In [1]:
import pyam
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("/home/patwuch/projects/ssp_rcp/data/raw/AR6_統計降尺度_日資料_臺南市_平均溫/GWL1.5/AR6_統計降尺度_日資料_臺南市_平均溫_ssp585_IPSL-CM6A-LR_2026.csv")

In [6]:
import os

In [13]:
len(os.listdir("/home/patwuch/projects/ssp_rcp/data/raw/AR6_統計降尺度_日資料_臺南市_平均溫/historical/"))

1540

In [15]:
len(os.listdir("/home/patwuch/projects/ssp_rcp/data/raw/AR6_統計降尺度_日資料_臺南市_平均溫/ssp245/"))

2236

In [ ]:
df = pyam.IamDataFrame(data="/home/patwuch/projects/gee_dengue_ml/tutorial_data.csv")

[INFO] 01:38:23 - pyam.core: Reading file /home/patwuch/projects/gee_dengue_ml/tutorial_data.csv
[INFO] 01:38:23 - pyam.core: Reading file /home/patwuch/projects/gee_dengue_ml/tutorial_data.csv


In [17]:
import os
import glob
import pandas as pd
import xarray as xr
import numpy as np

base_dir = "GWL"
variables = ["prec", "temp_min", "temp_max", "temp_avg"]
gwls = ["GWL1.5", "GWL2.0", "GWL3.0", "GWL4.0"]

all_datasets = []

for var in variables:
    for gwl in gwls:
        folder = os.path.join(base_dir, var, gwl)
        csv_files = glob.glob(os.path.join(folder, "*.csv"))
        
        for f in csv_files:
            fname = os.path.basename(f)
            # Extract SSP, model, year from filename, e.g. "_ssp245_MIROC6_2035.csv"
            parts = fname.split("_")
            ssp = next((p for p in parts if p.startswith("ssp")), None)
            model = parts[parts.index(ssp) + 1] if ssp else "unknown"
            year = [p for p in parts if p.isdigit()]
            year = year[0] if year else "0000"

            df = pd.read_csv(f)
            lon = np.unique(df["LON"])
            lat = np.unique(df["LAT"])

            # reshape from tabular to 2D grid (lat, lon)
            nlon = len(lon)
            nlat = len(lat)

            # Extract daily columns (YYYYMMDD)
            time_cols = [c for c in df.columns if c not in ["LON", "LAT"]]
            times = pd.to_datetime(time_cols, format="%Y%m%d")

            # Pivot table
            df_pivot = df.pivot(index="LAT", columns="LON")

            # Build a 3D array: [time, lat, lon]
            data = np.empty((len(times), nlat, nlon))
            for i, t in enumerate(time_cols):
                grid = df.pivot_table(values=t, index="LAT", columns="LON").values
                data[i, :, :] = grid

            da = xr.DataArray(
                data,
                dims=("time", "lat", "lon"),
                coords={"time": times, "lat": lat, "lon": lon},
                name=var,
                attrs={"ssp": ssp, "model": model, "gwl": gwl, "year": year}
            )

            all_datasets.append(da)

# Combine into one big dataset
ds = xr.merge(all_datasets)

# Optional: make gwl, ssp, model into coordinates (not just attributes)
ds = ds.expand_dims({
    "gwl": list(set([da.attrs["gwl"] for da in all_datasets])),
    "ssp": list(set([da.attrs["ssp"] for da in all_datasets])),
    "model": list(set([da.attrs["model"] for da in all_datasets]))
})

# Save to NetCDF
ds.to_netcdf("combined_GWL_SSP_daily.nc")


ModuleNotFoundError: No module named 'xarray'

In [13]:
df.model

['AIM/CGE 2.1',
 'GENeSYS-MOD 1.0',
 'IEA World Energy Model 2017',
 'IMAGE 3.0.1',
 'MESSAGEix-GLOBIOM 1.0',
 'POLES CD-LINKS',
 'REMIND-MAgPIE 1.7-3.0',
 'WITCH-GLOBIOM 4.4']